# Exercise 2: Genetic Algorithm
The genetic algorithm follows the basic structure of an evolutionary algorithm:
1. Generate an initial population.
2. Evalute the fitness of each individual of the population.
3. Select the best individuals from the population.
4. Generate offspring from the population and apply mutations.

&rarr; Repeat Step 2. to 4. until convergence or maximum number of generations

However, there a number of modifications for the Genetic Algorithm, which we will explore in this exercise.


## Import libraries

In [1]:
import numpy as np

%run assets/utility.py
np.set_printoptions(precision=4, suppress=True) # for better printing

## Chromosome

For the Genetic Algorithm, we need a phenotype encoding. For this exercise, we choose a discrete binary representation of our phenotype.

---
### **Assignment**
Implement the `generate_initial_population`-function. It should return a population with the following shape:

|   | Chromosome 1   | Chromosome 2   | $\cdots$   | Chromosome n   |
|:---:|:---:|:---:|:---:|:---:|
| Candidate 1   | $\dots$  | $\dots$  |  $\dots$ |  $\dots$ |
| Candidate 2  | $\dots$  |  $\dots$ | $\dots$  |  $\dots$ |
| $\vdots$  | $\dots$ | $\dots$  | $\dots$  | $\dots$  |
| Candidate m  |  $\dots$ | $\dots$  |  $\dots$ | $\dots$  |

---

In [2]:
def generate_initial_population(population_size, chromosome_length):
    population = ...
    return population

In [3]:
# Solution 1:
def generate_initial_population(population_size, chromosome_length):
    population = np.zeros((population_size, chromosome_length))
    
    chromosome = np.random.randint(0, 2, size=(chromosome_length,))
    for i in range(population_size):
        np.random.shuffle(chromosome) # shuffles in-place
        population[i,:] = chromosome
        
    return population

In [4]:
# Solution 2:
def generate_initial_population(population_size, chromosome_length):
    binary_values = [0, 1]
    probabilities = [0.5, 0.5]
    population = np.random.choice(binary_values, size=(population_size, chromosome_length), p=probabilities)

    return population

In [5]:
# Solution 3:
def generate_initial_population(population_size, chromosome_length):
    population = np.random.randint(0, 2, size=(population_size, chromosome_length))
    return population

In [6]:
# Test your function:
population_size = 8
chromosome_length = 5

population = generate_initial_population(population_size, chromosome_length)
print(f"Population: {population.shape}\n{population}")

Population: (8, 5)
[[0 0 1 0 0]
 [1 0 1 0 1]
 [0 1 1 0 0]
 [0 1 1 0 1]
 [1 0 0 1 0]
 [1 1 0 1 1]
 [0 1 0 0 1]
 [0 1 1 0 1]]


## Step 3: Select the best individuals from population with Tournament Selection

For selecting the parents of the next generation or the surviving individuals (if elitism is enabled), we use tournament selection.

### Steps for tournament selection:
1. Pick randomly $k$ individuals from the population.
2. Choose the individual with the highest fitness and make a copy.
3. Put all individuals back in the population.

---
### **Assignment**
Implement the `tournament_selection`-function as described above and in the lecture.

Ensure that the shape of the selected individual is fitting. For example, reshape an array with shape `(12,)` to `(1, 12)` with
```python
reshaped_array = array.reshape(1, -1)
```

---

In [7]:
def tournament_selection(population, fitness, tournament_size):
    best_individual_from_tournamet = ...
    return best_individual_from_tournamet

In [8]:
# Solution 1:
def tournament_selection(population, fitness, tournament_size):
    indices_list = np.random.choice(len(population), tournament_size, replace=False)
    chosen_parents = np.array([population[i] for i in indices_list])
    parent_fitnesses = np.array([fitnesses[i] for i in indices_list])
    max_fitness_idx = np.argmax(parent_fitnesses)

    return chosen_parents[max_fitness_idx].reshape(1, -1)

In [9]:
# Solution 2:
def tournament_selection(population, fitnesses, tournament_size):
    # Choose tournament participants
    indices = np.random.choice(population.shape[0], tournament_size, replace=False)
    
    # Evaluate winner
    contenders_fitness = fitnesses[indices]
    max_contenders_fitness_index = np.argmax(contenders_fitness)
    winner_index = indices[max_contenders_fitness_index]
    
    winner = population[winner_index,:]
    
    return winner.reshape(1, -1)

In [10]:
# Test your function
population_size = 8
chromosome_length = 12

population = np.random.randint(0, 2, size=(population_size, chromosome_length))
fitness = np.random.uniform(size=(population_size,))
tournament_size = 8 # we can only test for the full population, otherwise you need to add prints in your function.

winner = tournament_selection(population, fitness, tournament_size)

print(f"Fitness:\n{fitness} at {np.argmax(fitness)}")
print(f"Population:\n{population}")

index = search_index_in_columns(population, winner)
print(f"Tournament winner: {winner} at {index}")
print(f"Tournament winner shape: {winner.shape}")

Fitness:
[0.6914 0.4634 0.1657 0.4586 0.7359 0.5531 0.08   0.3294] at 4
Population:
[[1 0 1 0 0 1 1 0 0 0 1 1]
 [1 0 0 1 1 0 1 1 1 0 1 0]
 [0 1 0 0 1 1 0 0 0 1 0 0]
 [1 0 0 0 0 0 1 0 0 0 1 0]
 [1 0 1 1 1 0 0 1 1 0 1 1]
 [0 1 0 0 0 0 0 0 1 0 0 1]
 [0 1 1 1 0 1 0 1 0 0 1 1]
 [1 1 0 0 0 1 1 0 1 0 0 1]]
Tournament winner: [[1 0 1 1 1 0 0 1 1 0 1 1]] at 4
Tournament winner shape: (1, 12)


## Step 4a: Crossover

Crossover tries to emulate reproduction, which we can see in nature. With crossover, we can recombine the genetic material to form a new individual offspring. In this exercise, we implement a simple one point reciprocal crossover. As demonstrated in the lecture:

<img src="assets/one-point-crossover.png" width="550">

---
### **Assignment**
Implement the `stochastic_one_point_crossover`-function.
1. Randomly select a index for slicing the chromosomes.
2. Recombine the material two generate two unique chromosomes by interchanging the segments as demonstrated in the figure.
3. Add a probability evaluation. Generate with the a probability of the parameter `cross_probability` the crossovered offspring, otherwise return the parents as offspring.

### **Hint**
You can slice an array by using the indexing operation in NumPy.

```python
array_slice = array_1[0:index]
```
Be careful, we are working with column vectors here. We are therefore always working with the first row (index 0).

```python
array_slice = array_1[0, 0:index]
```

You can merge two slices by using `np.concatenate` or `np.hstack` (`np.vstack` does not work here). Debugging your code might require writing some prints in the function. After you ensured, that the code is working as desired, do not forget to remove the prints afterwards.

---

In [11]:
def stochastic_one_point_crossover(parent_1, parent_2, cross_probability):
    ...

In [12]:
# Solution 1:
def stochastic_one_point_crossover(parent_1, parent_2, crossover_probability):
    if np.random.rand() < crossover_probability:
        idx = np.random.randint(1, parent_1.shape[1])
        
        child_1 = np.concatenate((parent_1[0, :idx], parent_2[0, idx:]))
        child_2 = np.concatenate((parent_2[0, :idx], parent_1[0, idx:]))
        
        return child_1, child_2
    else:
        child_1 = parent_1
        child_2 = parent_2
        
        return child_1, child_2

In [13]:
# Solution 2:
def stochastic_one_point_crossover(parent_1, parent_2, crossover_probability):
    if np.random.rand() >= crossover_probability:
        return parent_1, parent_2
    
    idx = np.random.randint(low=1, high=parent_1.shape[1])
    
    child_1 = np.concatenate((parent_1[0, :idx], parent_2[0, idx:]))
    child_2 = np.concatenate((parent_2[0, :idx], parent_1[0, idx:]))
    
    child_1 = child_1.reshape(1, -1)
    child_2 = child_2.reshape(1, -1)
    
    return child_1, child_2

In [14]:
# Test your function:
chromosome_length = 6
crossover_probability = 1.0

parent_1 = np.random.randint(0, 2, size=(1, chromosome_length))
parent_2 = np.random.randint(0, 2, size=(1, chromosome_length))

child_1, child_2 = stochastic_one_point_crossover(parent_1, parent_2, crossover_probability)

print(f"Parent 1: {parent_1}\nParent 2: {parent_2}\n")
print(f"Child 1: {child_1}\nChild 2: {child_2}\n")

print(f"Parent 1 shape: {parent_1.shape}\nParent 2 shape: {parent_2.shape}\n")
print(f"Child 1 shape: {child_1.shape}\nChild 2 shape: {child_2.shape}\n")

Parent 1: [[0 0 0 1 1 1]]
Parent 2: [[0 1 0 1 1 0]]

Child 1: [[0 1 0 1 1 0]]
Child 2: [[0 0 0 1 1 1]]

Parent 1 shape: (1, 6)
Parent 2 shape: (1, 6)

Child 1 shape: (1, 6)
Child 2 shape: (1, 6)



## Step 4b: Mutation

In evolution, we can regularly observe mutations of gens (e.g. due to environmental factors). These mutations can result in the discovery of new solutions.

---
### **Assignment**
Implement the `mutate_genotypes`-function. Each chromosome of each member of the offspring population should change their value with `mutation_probability`.

---

In [15]:
def mutate_genotypes(population, mutation_probability):
    ...

In [16]:
# Solution 1:
def mutate_genotypes(population, mutation_probability):
    mutated_population = population.copy()

    for i in range(len(population)):
        for j in range(len(population[i])):
            if np.random.rand() < mutation_probability:
                if mutated_population[i][j] == 0:
                    mutated_population[i][j] = 1
                else:
                    mutated_population[i][j] = 0

    return mutated_population

In [17]:
# Solution 2:
def mutate_genotypes(population, mutation_probability):
    # Create a boolean mask where True indicates potential mutation sites
    mutation_mask = np.random.rand(*population.shape) < mutation_probability
    
    # Flip the bits where the mask is True
    mutated_population = population ^ mutation_mask

    return mutated_population

In [18]:
# Test your function:
mutation_probability = 0.05
population_size = 10
chromosome_length = 15

population = np.random.randint(0, 2, size=(population_size, chromosome_length))

mutated_population = mutate_genotypes(population, mutation_probability)

print(f"Original population:\n{population}\n")
print("Mutated population:")
highlight_mutations(population, mutated_population)
print("--> (mutations are highlighted)")

Original population:
[[1 0 0 1 0 1 0 1 0 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 0 1 0 1 0 1 1 1 0 1 0 0 0 0]
 [1 0 1 1 0 1 1 0 0 1 0 1 0 1 0]
 [0 0 0 0 1 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 1 1 0 1 1 1 0 0 0 1 0 0 1]
 [1 1 1 0 1 1 1 1 0 0 1 1 1 1 0]
 [0 1 1 1 1 0 0 0 0 1 0 0 1 1 0]]

Mutated population:
[[1 0 0 1 0 1 0 1 1 0 1 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 0 1 0 1 0 1 1 1 0 1 0 0 0 0]
 [1 0 1 1 0 1 1 0 0 1 1 1 0 1 0]
 [0 0 0 0 1 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 1 0 1 0]
 [0 1 0 0 0 1 1 1 1 0 0 1 0 0 0]
 [0 0 1 1 0 1 1 1 0 0 0 1 0 0 1]
 [1 1 1 0 1 1 1 1 0 0 1 1 1 1 0]
 [0 0 1 1 1 0 0 0 0 1 0 0 1 1 0]]
--> (mutations are highlighted)
